## Combbine all diagnosis

This method take diagnosis from images, clinical, and diagnosis sheet, and creates one ground truth (where all three agree) and one majority vote (where two agree) diagnosis files. 

In [7]:
# 이 셀만 실행시키면 된다
import pandas as pd
import math

clinical = pd.read_csv("D:/Desktop/aicoss-model/ADSP_PHC_COGN.csv")
#this file is the metadata file that one can get from downloading MRI images from ADNI
#img = pd.read_csv("INSERT_METADATA_FILE_HERE.csv")
comb = pd.read_csv("D:/Desktop/aicoss-model/DXSUM.csv")[["RID", "PTID" , "PHASE"]]

i=pd.read_csv("D:/Desktop/aicoss-model/DXSUM.csv")[["PTID" ,"DIAGNOSIS", "PHASE"]]
i=i.rename(columns = {"PTID":"Subject"}).rename(columns = { "DIAGNOSIS":"Group"}).rename(columns={"PHASE":"Phase"})
i = i.drop_duplicates()
i['Group'] = i['Group'] - 1

# generate csv
i[["Subject", "Group", "Phase"]].to_csv("D:/Desktop/aicoss-model/preprocessed/ground_truth.csv")

In [ ]:
##### 아래 과정 무시 #####

In [ ]:


def read_diagnose(file_path: str = 'DXSUM_PDXCONV_ADNIALL.csv', verbose=False):
    # Read diagnostic summary
    diagnostic_summary = pd.read_csv(file_path, index_col='PTID')
    diagnostic_summary = diagnostic_summary.sort_values(by=["update_stamp"], ascending=True)
    # Create dictionary
    diagnostic_dict: dict = {}
    for key, data in diagnostic_summary.iterrows():
        # Iterate for each row of the document
        phase: str = data['PHASE']
        diagnosis: float = -1.
        diagnosis = data['DIAGNOSIS']
       
        # Update dictionary
        if not math.isnan(diagnosis):
            diagnostic_dict[key] = diagnosis
    if verbose:
        print_diagnostic_dict_summary(diagnostic_dict)
    return diagnostic_dict


def print_diagnostic_dict_summary(diagnostic_dict: dict):
    print(f"Number of diagnosed patients: {len(diagnostic_dict.items())}\n")
    n_NL = 0
    n_MCI = 0
    n_AD = 0
    for (key, data) in diagnostic_dict.items():
        if data == 1:
            n_NL += 1
        if data == 2:
            n_MCI += 1
        if data == 3:
            n_AD += 1
    print(f"Number of NL patients: {n_NL}\n"
          f"Number of MCI patients: {n_MCI}\n"
          f"Number of AD patients: {n_AD}\n")

In [3]:
d = read_diagnose("D:/Desktop/attention_model/DXSUM.csv")
print_diagnostic_dict_summary(d)

Number of diagnosed patients: 17

Number of NL patients: 6
Number of MCI patients: 5
Number of AD patients: 6



In [4]:
new = pd.DataFrame.from_dict(d, orient='index').reset_index()

In [5]:
clinical.head()

,RID,PTID,SUBJID,PHASE,VISCODE,VISCODE2,EXAMDATE,PHC_Visit,PHC_Age_Cognition,PHC_Diagnosis,...,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter,update_stamp
0,2,011_S_0002,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,74.439425,1.0,...,0.337,0.372,1.0,0.253,0.301,1.0,0.907,0.703,0.0,2025-05-19 11:07:30.0
1,2,011_S_0002,ADNI_011_S_0002,ADNI1,m06,m06,2006-03-06,2,74.929500,1.0,...,0.347,0.381,1.0,0.509,0.316,1.0,-0.461,0.435,1.0,2025-05-19 11:07:30.0
2,2,011_S_0002,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,3,77.407255,1.0,...,0.511,0.383,1.0,0.509,0.316,1.0,0.907,0.703,0.0,2025-05-19 11:07:30.0
3,2,011_S_0002,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,4,79.477070,1.0,...,0.057,0.330,1.0,0.596,0.355,1.0,0.148,0.558,1.0,2025-05-19 11:07:30.0
4,2,011_S_0002,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,5,80.468172,1.0,...,-0.018,0.347,1.0,0.413,0.389,1.0,0.148,0.558,1.0,2025-05-19 11:07:30.0


In [6]:
clinical["year"] = clinical["EXAMDATE"].str[:4]

In [7]:
clinical["Subject"] = clinical["SUBJID"].str.replace("ADNI_", "").str.replace("s", "S")

In [8]:
c = comb.merge(clinical, on = ["RID", "PHASE"])

In [9]:
c = c.drop("Subject", axis =1)

In [ ]:
c = c.rename(columns = {"PTID":"Subject"}).rename(columns = {"PHC_Diagnosis":"DX"})

In [14]:
c["DX"] = c["DX"] -1
c.columns

Index(['RID', 'PTID_x', 'PHASE', 'PTID_y', 'SUBJID', 'VISCODE', 'VISCODE2',
       'EXAMDATE', 'PHC_Visit', 'PHC_Age_Cognition', 'DX', 'PHC_Sex',
       'PHC_Race', 'PHC_Ethnicity', 'PHC_Education', 'PHC_MEM', 'PHC_MEM_SE',
       'PHC_MEM_PreciseFilter', 'PHC_EXF', 'PHC_EXF_SE',
       'PHC_EXF_PreciseFilter', 'PHC_LAN', 'PHC_LAN_SE',
       'PHC_LAN_PreciseFilter', 'PHC_VSP', 'PHC_VSP_SE',
       'PHC_VSP_PreciseFilter', 'update_stamp', 'year'],
      dtype='object')

In [ ]:
img["year"] = img["Acq Date"].str[5:].str.replace("/", "")

In [ ]:
img = img.replace(["CN", "MCI", "AD"], [ 0, 1, 2])

In [24]:
new[0] = new[0].astype(int) -1

In [25]:
new = new.rename(columns = {"index":"Subject", 0:"GroupN"})

In [26]:
# m = new.merge(c, on = "Subject", how = "outer").merge(img, on = "Subject", how = "outer")
m = new
new.columns

Index(['Subject', 'GroupN'], dtype='object')

In [ ]:
m[["GroupN", "DX", "Group"]]

In [ ]:
m = m[["Subject", "GroupN", "Group", "DX", "Phase"]].drop_duplicates()

In [ ]:
m = m.dropna(subset = ["GroupN", "Group", "DX"], how="all").drop_duplicates()
m

In [ ]:
m.loc[m["DX"].isna() & m["Group"].isna(), "Group"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]
m.loc[m["DX"].isna() & m["Group"].isna(), "DX"] = m.loc[m["DX"].isna() & m["Group"].isna(), "GroupN"]

In [ ]:
m1 = m[m["GroupN"] == m["Group"]]
m3 = m[m["GroupN"] == m["DX"]]
m4 = m[m["Group"] == m["DX"]]
m2 = m1[m1["Group"] == m1["DX"]]

In [ ]:
m1 = m1[["Subject", "GroupN", "Group", "DX", "Phase"]]
m1

In [ ]:
m1.loc[m1["DX"].isna(), "DX"] = m1.loc[m1["DX"].isna(), "Group"]

In [ ]:
m3 = m3[["Subject", "GroupN", "Group", "DX", "Phase"]]
m3

In [ ]:
m3.loc[m3["Group"].isna(), "Group"] = m3.loc[m3["Group"].isna(), "GroupN"]

In [ ]:
m4 = m4[["Subject", "GroupN", "Group", "DX", "Phase"]]
m4

In [ ]:
m4[m4["GroupN"] != m4["DX"]]

In [ ]:
m2[["Subject", "GroupN", "Group", "DX", "Phase"]]

In [ ]:
m5 = pd.concat([m1,m3,m4])
i = m5[m5["Group"] == m5["GroupN"]]
i = i[i["Group"] == i["DX"]]

In [ ]:
i = i.drop_duplicates()

In [ ]:
##### 여기까지 무시 #####

In [ ]:
##### 아래 diagnosis.csv를 만드는 과정은 구현하지 않았으나 모델 구현 부분에서도 사용되지 않는 것으로 확인 #####

In [ ]:
m.update(m5[~m5.index.duplicated(keep='first')])

In [ ]:
indexes = m.index

In [ ]:
#if none of the three diagnosis agree, then we set the value to -1
m["GROUP"] = -1

In [ ]:
for i in indexes:
    row = m.loc[i]
    if (row["GroupN"] == row["Group"]):
        val = row["GroupN"]
        
        m.loc[i, "GROUP"] = val
    elif (row["GroupN"] == row["DX"]):
        val = row["GroupN"]
        m.loc[i, "GROUP"] = val
        
    elif (row["Group"] == row["DX"]):
        val = row["Group"]
        m.loc[i, "GROUP"] = val
        

In [ ]:
m5 = m5[~m5.index.duplicated(keep='first')]
m5

In [ ]:
m[m["GROUP"] != -1]

In [ ]:
m[["Subject", "GroupN", "Group", "DX", "GROUP", "Phase"]].to_csv("diagnosis_full.csv")